In [5]:
from heisembergs import sparse_xxz_hamiltonian
import scipy.sparse as sparse
from scipy.optimize import curve_fit
import numpy as np
import os

cwd = os.path.normpath(os.getcwd())
cwd = cwd.split(os.sep)
find = cwd.index("LogCorrections")
newd = f"{os.sep}".join(cwd[:find+1])
os.chdir(newd)
import sunny.shannon as ashannon

$$H = (-1)^{\mathrm{aferro} + 1}\sum_n (\sigma_{n}^x \sigma_{n+1}^x + \sigma_{n}^y \sigma_{n+1}^y) + \Delta \sum_n \sigma_{n}^z \sigma_{n+1}^z$$

In [ ]:
def get_numerical_max_amps(N_arr, antiferro, delta, return_energy=False, return_fnum=False, return_cnum=False, comparison_tol=1e-8, spin_penalty_factor=10, print_mode=False):
    max_prob_num_arr = np.zeros_like(N_arr, dtype=float)
    if return_energy: Egs_num_arr = np.zeros_like(N_arr, dtype=float)
    if return_fnum: n_fermions_num_arr = np.zeros_like(N_arr)
    if return_cnum: n_max_conf_num_arr = np.zeros_like(N_arr)
    for i, N in enumerate(N_arr):
        if print_mode: print(f"\rNum: N = {N}", end="")
        H = sparse_xxz_hamiltonian(delta, antiferro, N, spin_penalty_factor)
        evals, evecs = sparse.linalg.eigsh(H, which="SA", k=1, tol=1e-8)
        if return_energy: Egs_num_arr[i] = evals[0]
        amps = np.abs(evecs[:, 0])**2
        max_amp_ind = np.argmax(amps)
        max_prob_num_arr[i] = amps[max_amp_ind]
        if return_fnum: n_fermions_num_arr[i] = np.sum(np.array(list(f"{max_amp_ind:0{N}b}"), dtype=int))
        if return_cnum: n_max_conf_num_arr[i] = np.sum(np.isclose(amps, amps[max_amp_ind], atol=comparison_tol))
    to_return = [max_prob_num_arr]
    if return_energy: to_return.append(Egs_num_arr)
    if return_fnum: to_return.append(n_fermions_num_arr)
    if return_cnum: to_return.append(n_max_conf_num_arr)
    return np.squeeze(to_return)

### Antiferromagnetic, $\Delta = 0$

$H = -\sum_n (\sigma_{n}^x \sigma_{n+1}^x + \sigma_{n}^y \sigma_{n+1}^y)$

In [ ]:
def get_analytical_xx_max_amps(N_arr, return_energy=False, return_fnum=False, return_cnum=False, comparison_tol=1e-8, print_mode=False):
    prev_eps_ashannon = ashannon.epsilon
    ashannon.epsilon = comparison_tol
    max_prob_an_arr = np.zeros_like(N_arr, dtype=float)
    if return_energy: Egs_an_arr = np.zeros_like(N_arr, dtype=float)
    if return_fnum: n_fermions_an_arr = np.zeros_like(N_arr)
    if return_cnum: n_max_conf_an_arr = np.zeros_like(N_arr)
    for i, N in enumerate(N_arr):
        if print_mode: print(f"\rAn:  N = {N}", end="")
        if return_energy: Egs_an_arr[i] = ashannon.ground_state_energy(N)
        max_probs_obj = ashannon.max_probs(N)
        max_prob_an_arr[i] = max_probs_obj[0].prob
        if return_fnum: n_fermions_an_arr[i] = len(max_probs_obj[0].conf)
        if return_cnum: n_max_conf_an_arr[i] = len(max_probs_obj)
    ashannon.epsilon = prev_eps_ashannon
    to_return = [max_prob_an_arr]
    if return_energy: to_return.append(Egs_an_arr)
    if return_fnum: to_return.append(n_fermions_an_arr)
    if return_cnum: to_return.append(n_max_conf_an_arr)
    return np.squeeze(to_return)

def xx_num_an_comp_debrief(N_range, comparison_tol=1e-6, ndecimals=5, spin_penalty_factor=10, print_mode=False):
    N_arr = np.arange(*N_range, 2)
    max_prob_num_arr, Egs_num_arr, n_fermions_num_arr, n_max_conf_num_arr = get_numerical_max_amps(N_arr, False, 0, *[True]*3, comparison_tol, spin_penalty_factor, print_mode)
    max_prob_an_arr, Egs_an_arr, n_fermions_an_arr, n_max_conf_an_arr = get_analytical_xx_max_amps(N_arr, *[True]*3, comparison_tol, print_mode)
    print("\r-------------------------------------------------------------------------------------------------------")
    for i, N in enumerate(N_arr):
        N_str = f"N = {N}"
        first_line = "\t".join([N_str, "Num:", f"Egs = {Egs_num_arr[i]:.0{ndecimals}f}", f"max p = {max_prob_num_arr[i]:.0{ndecimals}f}", f"n_ferm = {n_fermions_num_arr[i]:.0{ndecimals}f}", f"n_conf = {n_max_conf_num_arr[i]:.0{ndecimals}f}"])
        second_line = "\t".join([" "*len(N_str), "An: ", f"Egs = {Egs_an_arr[i]:.0{ndecimals}f}", f"max p = {max_prob_an_arr[i]:.0{ndecimals}f}", f"n_ferm = {n_fermions_an_arr[i]:.0{ndecimals}f}", f"n_conf = {n_max_conf_an_arr[i]:.0{ndecimals}f}"])
        print(first_line)
        print(second_line)
        print("\r-------------------------------------------------------------------------------------------------------")

### Even sites

In [ ]:
N_range = [4, 20]
spin_penalty_factor = 0 # Default = 10
comparison_tol = 1e-6
ndecimals = 5
print_mode = True

xx_num_an_comp_debrief(N_range, comparison_tol, ndecimals, spin_penalty_factor, print_mode=print_mode)

-------------------------------------------------------------------------------------------------------
N = 4	Num:	Egs = -5.65685	max p = 0.25000	n_ferm = 2.00000	n_conf = 2.00000
     	An: 	Egs = -2.00000	max p = 0.25000	n_ferm = 3.00000	n_conf = 4.00000
-------------------------------------------------------------------------------------------------------
N = 6	Num:	Egs = -8.00000	max p = 0.12500	n_ferm = 3.00000	n_conf = 2.00000
     	An: 	Egs = -4.00000	max p = 0.12500	n_ferm = 3.00000	n_conf = 2.00000
-------------------------------------------------------------------------------------------------------
N = 8	Num:	Egs = -10.45250	max p = 0.06250	n_ferm = 4.00000	n_conf = 2.00000
     	An: 	Egs = -4.82843	max p = 0.04553	n_ferm = 5.00000	n_conf = 8.00000
-------------------------------------------------------------------------------------------------------
N = 10	Num:	Egs = -12.94427	max p = 0.03125	n_ferm = 5.00000	n_conf = 2.00000
      	An: 	Egs = -6.47214	max p = 0.03125	n_ferm

### Odd sites

In [ ]:
N_range = [7, 19]
spin_penalty_factor = 10 # Default = 10
comparison_tol = 1e-6
ndecimals = 5
print_mode = False

xx_num_an_comp_debrief(N_range, comparison_tol, ndecimals, spin_penalty_factor, print_mode=print_mode)

-------------------------------------------------------------------------------------------------------
N = 7	Num:	Egs = -8.98792	max p = 0.06626	n_ferm = 3.00000	n_conf = 7.00000
     	An: 	Egs = -4.49396	max p = 0.06626	n_ferm = 3.00000	n_conf = 7.00000
-------------------------------------------------------------------------------------------------------
N = 9	Num:	Egs = -11.51754	max p = 0.03106	n_ferm = 4.00000	n_conf = 9.00000
     	An: 	Egs = -5.75877	max p = 0.03106	n_ferm = 5.00000	n_conf = 9.00000
-------------------------------------------------------------------------------------------------------
N = 11	Num:	Egs = -14.05335	max p = 0.01476	n_ferm = 5.00000	n_conf = 11.00000
      	An: 	Egs = -7.02667	max p = 0.01476	n_ferm = 5.00000	n_conf = 11.00000
-------------------------------------------------------------------------------------------------------
N = 13	Num:	Egs = -16.59246	max p = 0.00707	n_ferm = 6.00000	n_conf = 13.00000
      	An: 	Egs = -8.29623	max p = 0.00707	

### $\Delta = -1$

# Fit comparison

In [ ]:
N_range = [7, 19]
delta = -1
antiferro = False
spin_penalty_factor = 20 # Default = 10
print_mode = True

initials = [0.1, 0.1, 0.1]

def to_fit_renyi_inf(N, alpha, beta, gamma):
    return alpha*N + beta*np.log(N) + gamma

# Numerical
N_arr = np.arange(*N_range, 2)
max_prob_num_arr = np.zeros_like(N_arr, dtype=float)
for i, N in enumerate(N_arr):
    if print_mode: print(f"\rNum: N = {N}", end="")
    H = sparse_xxz_hamiltonian(delta, antiferro, N, spin_penalty_factor)
    evals, evecs = sparse.linalg.eigsh(H, which="SA", k=1, tol=1e-8)
    amps = np.abs(evecs[:, 0])**2
    max_amp_ind = np.argmax(amps)
    max_prob_num_arr[i] = amps[max_amp_ind]

regression_num = curve_fit(to_fit_renyi_inf, N_arr, -np.log(max_prob_num_arr), p0=initials)
popt_num = regression_num[0]
errors_num = np.sqrt(np.diag(regression_num[1]))

# Analytical
if np.isclose(delta, 0):
    max_prob_an_arr = np.zeros_like(N_arr, dtype=float)
    for i, N in enumerate(N_arr):
        if print_mode: print(f"\rAn:  N = {N}", end="")
        max_probs_obj = ashannon.max_probs(N)
        max_prob_an_arr[i] = max_probs_obj[0].prob

    regression_an = curve_fit(to_fit_renyi_inf, N_arr, -np.log(max_prob_an_arr), p0=initials)
    popt_an = regression_an[0]
    errors_an = np.sqrt(np.diag(regression_an[1]))
    print_analytical = True
else:
    print_analytical = False

print("")
print("∞-Rènyi entropy fit result")
print("Fit function: ɑ L + β log L + Ɣ\n")

print("Numerical:")
print(f"ɑ = {popt_num[0]} ± {errors_num[0]}")
print(f"β = {popt_num[1]} ± {errors_num[1]}")
print(f"Ɣ = {popt_num[2]} ± {errors_num[2]}")

if print_analytical:
    print("Analytical:")
    print(f"ɑ = {popt_an[0]} ± {errors_an[0]}")
    print(f"β = {popt_an[1]} ± {errors_an[1]}")
    print(f"Ɣ = {popt_an[2]} ± {errors_an[2]}")

Num: N = 17
∞-Rènyi entropy fit result
Fit function: ɑ L + β log L + Ɣ

Numerical:
ɑ = 0.6882790563214433 ± 0.0003747201229627027
β = -0.383178485775744 ± 0.004239344246137672
Ɣ = -0.5167480682965924 ± 0.005920358095263444
